In [1]:
from glob import glob
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd

In [10]:
from joblib import Parallel, delayed

In [3]:
import datasets

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
root_dir = './data/ecg_better_model_mental_stress'
target_dir = './data/bare_30s_mental_stress'

In [5]:
participants = glob(f'{root_dir}/*.csv')

In [6]:
sampling_rate = 1000
n_window = 30 * sampling_rate

In [7]:
window_shift_size = 0.25
step_size = int(window_shift_size * sampling_rate)

In [8]:
from tqdm import tqdm

In [11]:
def windowed(window_size):
    def inner(idx, data):
        if idx + window_size > len(data):
            return None
        window = data[idx:idx + window_size]
        return { 'window': window['signal'], 'label': window['label'].mode()[0] }
    return inner

In [14]:
with tqdm(total=len(participants)) as pbar:
    for participant in participants:
        pbar.set_description(f'Processing {Path(participant).name} -- Reading CSV...')
        df = pd.read_csv(participant)

        pbar.set_description(f'Processing {Path(participant).name} -- Creating Windows...')
        new = Parallel(n_jobs=8)(delayed(windowed(n_window))(start_idx, df) for start_idx in range(0, len(df)))

        del df

        pbar.set_description(f'Processing {Path(participant).name} -- To HF dataset...')
        ds = datasets.Dataset.from_list(
            new, 
            features=datasets.Features({
                'window': datasets.Sequence(datasets.Value('float64')),
                'label': datasets.ClassLabel(num_classes=2)
            })
        )

        pbar.set_description(f'Processing {Path(participant).name} -- To Parquet...')
        ds.to_parquet(f'{target_dir}/{Path(participant).name}')
        pbar.update(1)

Processing 30100.csv -- Creating Windows...:   0%|          | 0/127 [20:06<?, ?it/s]


BrokenProcessPool: A result has failed to un-serialize. Please ensure that the objects returned by the function are always picklable.